In [80]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import torch
import sys
import kaolin as kal
import matplotlib.pyplot as plt
import igl

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'nightstand'
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


In [81]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices_tensor = mesh.vertices.to(device)
faces_tensor = mesh.faces.to(device)

vertices = vertices_tensor.detach().cpu().numpy()
faces = faces_tensor.detach().cpu().numpy()
colors = mesh.vertex_normals.cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  6640
Number of faces:  8426


In [82]:
# Sample one point
sampled_pts = igl.random_points_on_mesh(1, vertices, faces)
print('Face:', sampled_pts[1])
pt = np.sum(sampled_pts[0][:, np.newaxis] * vertices[faces[sampled_pts[1]]], axis=0)[np.newaxis, :]
print('Point:', pt)
p = mp.plot(vertices, faces, return_plot=True)
p.add_points(pt, shading={"point_size": 0.5, "point_color": "green"})
plt.show()

Face: 3
Point: [[ 0.00790948  0.72697294 -0.07581344]]


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.36…

In [83]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

In [130]:
# Number of points to sample
N = int(vertices.shape[0] / 2)
# Sample points
sampled_pts = igl.random_points_on_mesh(N, vertices, faces)
barycenters = sampled_pts[0]
barycenters = barycenters[:,:,np.newaxis]
face_idx = faces[sampled_pts[1]]
pts = np.sum(barycenters * vertices[face_idx], axis = 1)
query_point = np.array([[0, 1, 0]])
p = mp.plot(vertices, faces, colors, return_plot=True)
p.add_points(pts, shading={"point_size": 0.1, "point_color": "green"})
p.add_points(query_point, shading={"point_size": 0.4, "point_color": "red"})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.36…

In [148]:
from sklearn.neighbors import NearestNeighbors
k = 4 
nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto')
nbrs.fit(pts)
distances, indices = nbrs.kneighbors(query_point)
nearesNeighbors = pts[indices.squeeze()]
v_box = np.vstack((query_point, nearesNeighbors))
f_box = np.array([[0, i] for i in range(1, v_box.shape[0])])
p = mp.plot(vertices, faces, colors, return_plot=True)
p.add_points(pts, shading={"point_size": 0.1, "point_color": "green"})
p.add_points(query_point, shading={"point_size": 0.2, "point_color": "red"})
p.add_points(nearesNeighbors, shading={"point_size": 0.2, "point_color": "blue"})
p.add_edges(v_box, f_box, shading={"line_color": "blue"})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.36…